# JSONファイルからRAG用のベクトルDBを作る

LangChain を使って RAG を試してみた #AI - Qiita
> https://qiita.com/tinymouse/items/4d359674f6b2494bb22d

LLMアプリケーション開発のためのLangChain 後編⑤ 外部ドキュメントのロード、分割及び保存 - qiita
> https://qiita.com/utanesuke/items/6efc03eca94f7de3b9cd#json-%E3%83%AD%E3%83%BC%E3%83%80%E3%83%BC


## install

In [ ]:
!pip install unsloth langchain langchain_community langchain-huggingface sentence-transformers transformers accelerate lancedb

In [ ]:
!pip install -U langchain langchain-community lancedb

In [ ]:
!conda install 

## imports

In [1]:
# -*- coding: utf-8 -*-
from unsloth import FastLanguageModel
import transformers
import torch

import json
from pathlib import Path
from typing import List, Dict, Any
import os
import math

import pyarrow as pa
import lancedb
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import numpy as np
import time

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## prepare tokenizer & model, settings

In [2]:

# ----------------------------
# Config
# ----------------------------
EMBED_MODEL = "./infloat_multilingual-e5-large/"
DB_PATH = "./LanceDB/rules.lancedb"
TABLE_NAME = "rules"
JSON_PATH = Path("./rspec_rules.json")
BATCH_SIZE = 256
USE_DEVICE = "cuda"

# モデルのロード
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = 40960,
    load_in_4bit = True,
    load_in_8bit = False,
)

pipe = transformers.pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    dtype=torch.float16
)

==((====))==  Unsloth 2025.9.11: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.422 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## prepare embedding model

In [3]:
# ----------------------------
# 1) Embedding モデル
# ----------------------------
emb_model = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={"device": USE_DEVICE},
    encode_kwargs={"normalize_embeddings": True},
)

def embed(text: str) -> List[float]:
    """E5 埋め込みを取得"""
    formatted = f"passage: {text}"
    emb = emb_model.embed_query(formatted)
    arr = np.asarray(emb, dtype=np.float32)
    return arr.tolist()

# LangChainのHuggingFacePipelineは非推奨の警告が出ることがあるため、
# 直接呼び出すか、必要な場合のみインポートしてください。
# ここではRAG部分で直接 generate を呼んでいるため llm 変数は必須ではありませんが、
# 念のため残す場合は transformers の pipeline をラップします。
# from langchain_huggingface import HuggingFacePipeline
# llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


## make TextSplitter

In [4]:
# ----------------------------
# 2) Splitter
# ----------------------------
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
)

## setup LanceDB

In [5]:
# ----------------------------
# 3) LanceDB 接続 & schema 作成
# ----------------------------
db = lancedb.connect(DB_PATH)

# すでにテーブルがあるか確認
if TABLE_NAME in db.table_names():
    print(f"テーブル {TABLE_NAME} は既に存在します。ロードします...")
    table = db.open_table(TABLE_NAME)
else:
    print(f"テーブル {TABLE_NAME} が存在しないため、新規作成します...")

    print("確認: 埋め込み次元を取得しています...")
    sample_dim = len(embed("this is a sample"))
    print("Embedding dim detected:", sample_dim)
    EMB_DIM = sample_dim

    schema = pa.schema([
        ("id", pa.string()),
        ("title", pa.string()),
        ("chunk", pa.string()),
        ("embedding", pa.list_(pa.float32(), list_size=EMB_DIM)), 
        ("url", pa.string()),
    ])

    table = db.create_table(TABLE_NAME, schema=schema, mode="overwrite")
    print(f"テーブル {TABLE_NAME} を作成しました。")

Document(metadata={'instruction': 3, 'output': 4}, page_content='passage: Q. SQL インジェクションの脆弱性の例を教えて  A. carmelogarcia の Simple Leave Manager In PHP With Source Code における SQL インジェクションの脆弱性が見つかっています。A flaw has been found in code-projects Simple Leave Manager 1.0. This vulnerability affects unknown code of the file /user.php. This manipulation of the argument table causes sql injection. Remote exploitation of the attack is possible. The exploit has been published and may be used. 該当する製品はcarmelogarcia\nSimple Leave Manager In PHP With Source Code 1.0になります。\n')


## load Json & make DB

In [6]:
# ----------------------------
# 4) JSON 読み込みとデータ投入
# ----------------------------
if JSON_PATH.exists():
    with JSON_PATH.open("r", encoding="utf-8") as f:
        data = json.load(f)

    rows_to_insert = []
    
    print("データ処理を開始します...")
    for item in data:
        title = item["title"]
        description = item["description"]
        url = item.get("url", "")

        base_text = f"{title}\n\n{description}"
        chunks = splitter.split_text(base_text)

        for idx, ch in enumerate(chunks):
            if ch.strip() == title:
                continue
            
            emb = embed(ch)

            rows_to_insert.append({
                "id": f"{url}#{idx}",
                "title": title,
                "chunk": ch,
                "embedding": emb,
                "url": url
            })
            
            # 【修正点2】 メモリ節約のため一定数たまったらinsertしてリストをクリアする
            if len(rows_to_insert) >= BATCH_SIZE:
                table.add(rows_to_insert)
                print(f"✓ {len(rows_to_insert)} 件追加しました")
                rows_to_insert = [] # リセット

    # ループ終了後に残っているデータを追加
    if rows_to_insert:
        table.add(rows_to_insert)
        print(f"✓ 残りの {len(rows_to_insert)} 件を追加しました")

else:
    print(f"警告: {JSON_PATH} が見つかりません。ダミーデータなしで検索に進みます。")

/tmp/ipykernel_29406/2349016760.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(


## define similer-text search

In [7]:
# ----------------------------
# 5) 検索関数
# ----------------------------
def search(query: str, k: int = 5):
    formatted = f"query: {query}"
    query_emb = emb_model.embed_query(formatted)

    results = (
        table.search(query_emb)
        .metric("cosine")
        .limit(k)
        .to_list()
    )
    return results

def build_context(docs: List[Dict[str, Any]]) -> str:
    """
    検索結果(docs)のリストを、LLMに渡すためのテキスト形式に整形する関数
    """
    parts = []
    for i, d in enumerate(docs):
        # 辞書からタイトルと本文を取得
        title = d.get("title", "No Title")
        chunk = d.get("chunk", "")
        
        # スコアの取得 (LanceDBのバージョンによってキー名が異なる場合への対応)
        # _distance は距離なので低いほうが類似度が高いですが、ここでは参考値として表示します
        score = d.get("_score", d.get("_distance", 0.0))
        
        # 整形フォーマット: [番号] (スコア) タイトル -> 本文
        part_text = f"Reference [{i+1}] (Metric: {score:.4f})\nTitle: {title}\nContent:\n{chunk}"
        parts.append(part_text)
    
    # 各ドキュメントを区切り線で結合して返す
    return "\n\n" + ("-" * 20) + "\n\n".join(parts)

## load DB

In [ ]:
# ----------------------------
# 6) LLM推論ヘルパー & 新フローの実装
# ----------------------------
def run_llm_inference(messages: List[Dict[str, str]], max_tokens: int = 256) -> str:
    """
    ストリーミングせずにLLMの出力を文字列として取得するヘルパー関数
    (Step 1: 脆弱性特定用)
    """
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )
    
    inputs = tokenizer(text, return_tensors="pt").to(USE_DEVICE)
    
    # generate時は勾配計算不要
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.3, # 特定タスクなのでランダム性を下げる
            top_p=0.95,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 入力プロンプト部分を除去して応答のみ抽出
    generated_ids = outputs[0][len(inputs.input_ids[0]):]
    decoded_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return decoded_text.strip()

def rag_answer_v2(code: str, k: int = 3):
    """
    3段階フローの実装:
    1. LLMで脆弱性名を特定
    2. その名前でRAG検索
    3. RAG情報を元に対策を出力
    """
    
    print("\n=== Step 1: LLMによる脆弱性診断を実行中... ===")
    
    # --- Step 1: 脆弱性の特定 (RAGなし) ---
    detect_system_prompt = """You are a specialized security code auditor.
Analyze the provided source code and identify the single most critical security vulnerability present.
Output ONLY the standard name of the vulnerability (e.g., "Cross-Site Scripting (XSS)", "SQL Injection", "OS Command Injection").
Do not provide explanations or code fixes yet. If no vulnerability is found, output "None".
"""
    
    detect_messages = [
        {"role": "system", "content": detect_system_prompt},
        {"role": "user", "content": f"Analyze this code:\n\n{code}"}
    ]
    
    # 推論実行（短く回答させる）
    detected_vulnerability = run_llm_inference(detect_messages, max_tokens=50)
    print(f"▶ 特定された脆弱性: {detected_vulnerability}")

    if "None" in detected_vulnerability:
        print("脆弱性は検出されませんでした。")
        return

    # --- Step 2: RAGで関連情報の取得 ---
    print(f"\n=== Step 2: 「{detected_vulnerability}」に関する情報を検索中... ===")
    
    # 元の質問文ではなく、特定された「脆弱性名」で検索をかける
    docs = search(detected_vulnerability, k=k)
    
    if not docs:
        print("※ 関連ドキュメントが見つかりませんでした。一般的な知識で回答します。")
        context = "No external documentation found."
    else:
        context = build_context(docs)
        # 検索結果のタイトルを表示（確認用）
        for i, d in enumerate(docs):
            print(f"  Found [{i+1}]: {d['title']}")

    # --- Step 3: 最終回答の生成 (RAGあり) ---
    print("\n=== Step 3: 対策と解説の生成中... ===\n")

    final_system_prompt = f"""You are a white hat hacker.
The user wants to fix a vulnerability identified as "{detected_vulnerability}".
Based on the provided "Reference Information" (RAG Context) and the source code, provide:
1. An explanation of why this code is vulnerable to {detected_vulnerability}.
2. Risks associated with it.
3. A corrected version of the code or specific mitigation steps.

# Guidelines
- Answer in JAPANESE (日本語).
- Use Markdown format.
- Prioritize the methods suggested in the Reference Information.
"""

    final_user_message = f"""
# Reference Information (Guidelines)
{context}

# Source Code
{code}
"""

    messages = [
        {"role": "system", "content": final_system_prompt},
        {"role": "user", "content": final_user_message}
    ]
    
    # ストリーミング用のテキスト準備
    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )

    from transformers import TextStreamer
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    
    # 最終生成
    model.generate(
        **tokenizer(text_input, return_tensors="pt").to(USE_DEVICE),
        max_new_tokens=2048,
        temperature=0.6,
        top_p=0.95,
        streamer=streamer,
    )




## infer qwen3 & embedding model

In [11]:
# ----------------------------
# 7) 実行
# ----------------------------
# 脆弱性を含むサンプルコード（元のコードと同じ）
c = """

"""

# 以前のように質問文(q)を渡す必要はなく、コード(c)を渡して診断させる
rag_answer_v2(c, k=5)

In [11]:
# ----------------------------
# 7) 実行
# ----------------------------
# 脆弱性を含むサンプルコード（元のコードと同じ）
c = """

"""

# 以前のように質問文(q)を渡す必要はなく、コード(c)を渡して診断させる
rag_answer_v2(c, k=5)

In [11]:
# ----------------------------
# 7) 実行
# ----------------------------
# 脆弱性を含むサンプルコード（元のコードと同じ）
c = """
<?php
// php_d100_roller.php
// Simple 100-sided dice roller application (single file)
// Usage: Place this file on your web server and open it in a browser.
// For local testing: run `php -S localhost:8000` and visit http://localhost:8000/php_d100_roller.php

session_start();

// Keep roll history in session (max 100 entries)
if (!isset($_SESSION['d100_history'])) {
    $_SESSION['d100_history'] = [];
}

$errors = [];
$results = [];
$total = 0;
$count = 1;

if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    // Get roll count (clamped between 1–100)
    $count = isset($_POST['count']) ? intval($_POST['count']) : 1;
    if ($count < 1) $count = 1;
    if ($count > 100) $count = 100;

    // Optional label
    $label = isset($_POST['label']) ? trim($_POST['label']) : '';

    // Perform rolls
    for ($i = 0; $i < $count; $i++) {
        $roll = random_int(1, 100);
        $results[] = $roll;
        $total += $roll;
    }

    // Add to history (newest first)
    $entry = [
        'time' => date('Y-m-d H:i:s'),
        'count' => $count,
        'label' => $label,
        'results' => $results,
        'total' => $total,
    ];

    array_unshift($_SESSION['d100_history'], $entry);
    if (count($_SESSION['d100_history']) > 100) {
        $_SESSION['d100_history'] = array_slice($_SESSION['d100_history'], 0, 100);
    }
}

$history = $_SESSION['d100_history'];
?>
<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>100-sided Dice Roller</title>
<style>
    body { font-family: system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", Arial; padding: 20px; }
    .box { max-width: 820px; margin: 0 auto; }
    input[type="number"] { width: 80px; }
    .badge { display:inline-block; padding:6px 10px; margin:4px; border-radius:6px; background:#eee; }
    .roll { font-weight:700; }
    .history { margin-top:20px; }
    .card { padding:12px; border:1px solid #ddd; border-radius:8px; margin-bottom:12px; }
    .muted { color:#666; font-size:0.9rem; }
</style>
</head>
<body>
<div class="box">
    <h1>100-sided Dice Roller</h1>
    <form method="post" action="<?php echo htmlspecialchars($_SERVER['PHP_SELF']); ?>">
    <label>Number of rolls (1–100): <input type="number" name="count" value="<?php echo htmlspecialchars($count); ?>" min="1" max="100"></label>
    &nbsp;
    <label>Label (optional): <input type="text" name="label" value=""></label>
    &nbsp;
    <button type="submit">Roll</button>
    </form>

    <?php if (!empty($results)): ?>
    <div class="card">
        <div class="muted">Timestamp: <?php echo htmlspecialchars($entry['time']); ?></div>
        <h2>Results</h2>
        <div>
        <?php foreach ($results as $i => $r): ?>
            <span class="badge roll">#<?php echo $i+1; ?>: <?php echo $r; ?></span>
        <?php endforeach; ?>
        </div>
        <p>Total: <strong><?php echo $total; ?></strong> / Average: <strong><?php echo count($results) ? round($total / count($results), 2) : 0; ?></strong></p>
        <?php if ($entry['label'] !== ''): ?><p>Label: <?php echo htmlspecialchars($entry['label']); ?></p><?php endif; ?>
    </div>
    <?php endif; ?>

    <div class="history">
    <h2>History (last <?php echo count($history); ?> rolls)</h2>
    <?php if (empty($history)): ?>
        <p class="muted">No rolls yet.</p>
    <?php else: ?>
        <?php foreach ($history as $idx => $h): ?>
        <div class="card">
            <div class="muted"><?php echo htmlspecialchars($h['time']); ?> — Rolls: <?php echo $h['count']; ?><?php if ($h['label'] !== ''): ?> — Label: <?php echo htmlspecialchars($h['label']); ?><?php endif; ?></div>
            <div style="margin-top:8px;">
            <?php foreach ($h['results'] as $i => $r): ?>
                <span class="badge"><?php echo $r; ?></span>
            <?php endforeach; ?>
            </div>
            <p style="margin-top:8px;">Total: <?php echo $h['total']; ?> / Average: <?php echo count($h['results']) ? round($h['total'] / count($h['results']), 2) : 0; ?></p>
        </div>
        <?php endforeach; ?>
    <?php endif; ?>
    </div>

    <div style="margin-top:20px;" class="muted">
    <p>Note: Uses <code>random_int(1, 100)</code> for cryptographically secure random number generation. The session keeps up to 100 history entries.</p>
    </div>
</div>
</body>
</html>
"""

# 以前のように質問文(q)を渡す必要はなく、コード(c)を渡して診断させる
rag_answer_v2(c, k=5)

In [11]:
# ----------------------------
# 7) 実行
# ----------------------------
# 脆弱性を含むサンプルコード（元のコードと同じ）
c = """

"""

# 以前のように質問文(q)を渡す必要はなく、コード(c)を渡して診断させる
rag_answer_v2(c, k=5)

In [11]:
# ----------------------------
# 7) 実行
# ----------------------------
# 脆弱性を含むサンプルコード（元のコードと同じ）
c = """

"""

# 以前のように質問文(q)を渡す必要はなく、コード(c)を渡して診断させる
rag_answer_v2(c, k=5)

In [11]:
# ----------------------------
# 7) 実行
# ----------------------------
# 脆弱性を含むサンプルコード（元のコードと同じ）
c = """

"""

# 以前のように質問文(q)を渡す必要はなく、コード(c)を渡して診断させる
rag_answer_v2(c, k=5)

In [11]:
# ----------------------------
# 7) 実行
# ----------------------------
# 脆弱性を含むサンプルコード（元のコードと同じ）
c = """

"""

# 以前のように質問文(q)を渡す必要はなく、コード(c)を渡して診断させる
rag_answer_v2(c, k=5)

In [11]:
# ----------------------------
# 7) 実行
# ----------------------------
# 脆弱性を含むサンプルコード（元のコードと同じ）
c = """

"""

# 以前のように質問文(q)を渡す必要はなく、コード(c)を渡して診断させる
rag_answer_v2(c, k=5)

In [11]:
# ----------------------------
# 7) 実行
# ----------------------------
# 脆弱性を含むサンプルコード（元のコードと同じ）
c = """

"""

# 以前のように質問文(q)を渡す必要はなく、コード(c)を渡して診断させる
rag_answer_v2(c, k=5)

In [11]:
# ----------------------------
# 7) 実行
# ----------------------------
# 脆弱性を含むサンプルコード（元のコードと同じ）
c = """

"""

# 以前のように質問文(q)を渡す必要はなく、コード(c)を渡して診断させる
rag_answer_v2(c, k=5)